In [18]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import numpy as np

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

In [19]:
data = pd.read_excel('../Stats_competition-/Teams_Data.xlsx')
display(data)

data = data.dropna(subset=['Opponent_WAB'])
print(data.columns)

feature_cols = data.columns.difference(['Team', 'Opponent', 'Team Score', 'Opponent Score', 'Team_TEAM','Team_CONF', 'Team_G','Opponent_TEAM','Opponent_CONF', 'Opponent_G'])
X = data[feature_cols]
y = data[['Team Score', 'Opponent Score']]

In [24]:
def train_and_select_best_models(X, y):
    models = {}
    for target in y.columns:
        y_target = y[target]
        X_train, X_test, y_train, y_test = train_test_split(X, y_target, test_size=0.2, random_state=42)

        candidate_models = {
            'RandomForest': RandomForestRegressor(n_estimators=100, random_state=42),
            'GradientBoosting': GradientBoostingRegressor(random_state=42),
            'LinearRegression': LinearRegression()
        }

        best_model = None
        best_score = float('inf')
        for name, model in candidate_models.items():
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
            rmse = mean_squared_error(y_test, y_pred, squared=False)
            print(f"{target} - {name}: RMSE = {rmse:.4f}")
            if rmse < best_score:
                best_score = rmse
                best_model = model

        models[target] = best_model
        
    return models

In [25]:
models = train_and_select_best_models(X, y)

Team Score - RandomForest: RMSE = 14.1992
Team Score - GradientBoosting: RMSE = 16.2793
Team Score - LinearRegression: RMSE = 10.1629
Opponent Score - RandomForest: RMSE = 7.5801
Opponent Score - GradientBoosting: RMSE = 14.8409
Opponent Score - LinearRegression: RMSE = 40.2279


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_reg

In [26]:
models

{'Team Score': LinearRegression(),
 'Opponent Score': RandomForestRegressor(random_state=42)}

In [27]:
def predict_and_simulate(models, X, n_simulations=10000):
    results = []

    for idx, row in X.iterrows():
        row_results = {'Index': idx}

        for target, model in models.items():
            score_pred = model.predict([row])[0]

            residual_std = 10 
            simulated_scores = np.random.normal(score_pred, residual_std, n_simulations)
            confidence_interval = (np.percentile(simulated_scores, 2.5), np.percentile(simulated_scores, 97.5))

            row_results[f'{target}_Predicted_Score'] = score_pred
            row_results[f'{target}_Confidence_Interval'] = confidence_interval

        results.append(row_results)

    return pd.DataFrame(results)

In [28]:
test_df = pd.read_excel('../Stats_competition-/Teams_Data_test.xlsx')

In [29]:
test_df

,Team,Opponent,Team_ADJOE,Team_ADJDE,Team_BARTHAG,Team_EFG_O,Team_EFG_D,Team_TOR,Team_TORD,Team_ORB,Team_DRB,Team_FTR,Team_FTRD,Team_2P_O,Team_2P_D,Team_3P_O,Team_3P_D,Team_ADJ_T,Team_WAB,Opponent_ADJOE,Opponent_ADJDE,Opponent_BARTHAG,Opponent_EFG_O,Opponent_EFG_D,Opponent_TOR,Opponent_TORD,Opponent_ORB,Opponent_DRB,Opponent_FTR,Opponent_FTRD,Opponent_2P_O,Opponent_2P_D,Opponent_3P_O,Opponent_3P_D,Opponent_ADJ_T,Opponent_WAB
0,Duke,Auburn,117.516,88.610,0.962610,55.352,40.7500,16.612,21.3540,33.6940,24.0250,26.7306,27.1620,56.4570,40.3400,36.1760,27.5270,68.0197,0.4543,123.7100,94.4110,0.957140,60.6400,45.0420,13.9290,16.0252,32.3123,25.4430,29.6264,30.6121,65.7100,42.8250,36.3730,33.9211,68.9139,3.0100
1,Duke,Louisville,117.516,88.610,0.962610,55.352,40.7500,16.612,21.3540,33.6940,24.0250,26.7306,27.1620,56.4570,40.3400,36.1760,27.5270,68.0197,0.4543,113.0540,97.9500,0.838645,52.1130,46.2570,18.5222,24.9500,36.2410,30.9220,33.9174,33.2172,61.6800,43.9330,28.7298,33.1179,70.1780,0.1953
2,North Carolina,Alabama,119.160,98.759,0.896518,54.666,50.2171,12.740,14.4315,25.9279,28.0113,43.2440,28.1810,53.5121,49.1136,37.7460,34.6237,73.0900,0.0463,121.7300,96.8330,0.932780,54.7640,45.5510,16.9132,13.3340,35.6500,28.7140,46.3230,38.6269,60.3140,46.2690,32.5194,29.5700,73.2700,1.8600
3,North Carolina,Georgia Tech,119.160,98.759,0.896518,54.666,50.2171,12.740,14.4315,25.9279,28.0113,43.2440,28.1810,53.5121,49.1136,37.7460,34.6237,73.0900,0.0463,109.6910,103.3127,0.664711,50.4179,47.8101,14.2370,14.6305,32.5121,25.6460,31.5224,37.4249,51.4173,44.9460,32.6192,35.3255,70.8470,-1.9194
4,N.C. State,Texas,110.285,98.355,0.787168,52.312,45.1450,15.157,22.1330,29.5192,33.5290,37.4990,30.0109,56.1670,48.8125,29.5284,26.0140,68.4171,-0.5940,115.0360,95.5190,0.894621,56.7290,42.6190,12.7400,19.4102,28.3216,27.1840,38.0910,28.7900,58.2340,38.2100,35.7900,33.6196,68.6160,0.3049
5,N.C. State,Florida State,110.285,98.355,0.787168,52.312,45.1450,15.157,22.1330,29.5192,33.5290,37.4990,30.0109,56.1670,48.8125,29.5284,26.0140,68.4171,-0.5940,107.6121,96.0240,0.786870,52.6108,44.8370,17.9186,25.4300,29.9181,30.4201,45.5270,40.4296,54.7980,49.4142,32.5194,26.4170,72.1200,0.1854


In [30]:
test_X = test_df[feature_cols] 

In [31]:
test_X

,Opponent_2P_D,Opponent_2P_O,Opponent_3P_D,Opponent_3P_O,Opponent_ADJDE,Opponent_ADJOE,Opponent_ADJ_T,Opponent_BARTHAG,Opponent_DRB,Opponent_EFG_D,Opponent_EFG_O,Opponent_FTR,Opponent_FTRD,Opponent_ORB,Opponent_TOR,Opponent_TORD,Opponent_WAB,Team_2P_D,Team_2P_O,Team_3P_D,Team_3P_O,Team_ADJDE,Team_ADJOE,Team_ADJ_T,Team_BARTHAG,Team_DRB,Team_EFG_D,Team_EFG_O,Team_FTR,Team_FTRD,Team_ORB,Team_TOR,Team_TORD,Team_WAB
0,42.8250,65.7100,33.9211,36.3730,94.4110,123.7100,68.9139,0.957140,25.4430,45.0420,60.6400,29.6264,30.6121,32.3123,13.9290,16.0252,3.0100,40.3400,56.4570,27.5270,36.1760,88.610,117.516,68.0197,0.962610,24.0250,40.7500,55.352,26.7306,27.1620,33.6940,16.612,21.3540,0.4543
1,43.9330,61.6800,33.1179,28.7298,97.9500,113.0540,70.1780,0.838645,30.9220,46.2570,52.1130,33.9174,33.2172,36.2410,18.5222,24.9500,0.1953,40.3400,56.4570,27.5270,36.1760,88.610,117.516,68.0197,0.962610,24.0250,40.7500,55.352,26.7306,27.1620,33.6940,16.612,21.3540,0.4543
2,46.2690,60.3140,29.5700,32.5194,96.8330,121.7300,73.2700,0.932780,28.7140,45.5510,54.7640,46.3230,38.6269,35.6500,16.9132,13.3340,1.8600,49.1136,53.5121,34.6237,37.7460,98.759,119.160,73.0900,0.896518,28.0113,50.2171,54.666,43.2440,28.1810,25.9279,12.740,14.4315,0.0463
3,44.9460,51.4173,35.3255,32.6192,103.3127,109.6910,70.8470,0.664711,25.6460,47.8101,50.4179,31.5224,37.4249,32.5121,14.2370,14.6305,-1.9194,49.1136,53.5121,34.6237,37.7460,98.759,119.160,73.0900,0.896518,28.0113,50.2171,54.666,43.2440,28.1810,25.9279,12.740,14.4315,0.0463
4,38.2100,58.2340,33.6196,35.7900,95.5190,115.0360,68.6160,0.894621,27.1840,42.6190,56.7290,38.0910,28.7900,28.3216,12.7400,19.4102,0.3049,48.8125,56.1670,26.0140,29.5284,98.355,110.285,68.4171,0.787168,33.5290,45.1450,52.312,37.4990,30.0109,29.5192,15.157,22.1330,-0.5940
5,49.4142,54.7980,26.4170,32.5194,96.0240,107.6121,72.1200,0.786870,30.4201,44.8370,52.6108,45.5270,40.4296,29.9181,17.9186,25.4300,0.1854,48.8125,56.1670,26.0140,29.5284,98.355,110.285,68.4171,0.787168,33.5290,45.1450,52.312,37.4990,30.0109,29.5192,15.157,22.1330,-0.5940


In [32]:
results_df = predict_and_simulate(models, test_X)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: 

In [33]:
df_combined = pd.concat([test_df, results_df.drop('Index', axis = 1)], axis = 1)

In [34]:
df_combined[['Team', 'Opponent','Team Score_Predicted_Score',
       'Team Score_Confidence_Interval', 'Opponent Score_Predicted_Score',
       'Opponent Score_Confidence_Interval']]

,Team,Opponent,Team Score_Predicted_Score,Team Score_Confidence_Interval,Opponent Score_Predicted_Score,Opponent Score_Confidence_Interval
0,Duke,Auburn,55.000000,"(34.83037428908654, 74.29454176321431)",78.66,"(59.05704225510345, 98.25232738339608)"
1,Duke,Louisville,96.802176,"(77.17922515368281, 116.58692270852146)",73.18,"(53.7570645001215, 92.23538362600029)"
2,North Carolina,Alabama,66.726569,"(47.448444148932296, 86.12624019304728)",77.60,"(57.454986632525625, 97.11397485418486)"
3,North Carolina,Georgia Tech,84.338049,"(64.65532049913325, 104.11011046911825)",65.73,"(46.22729595130238, 85.33924657193872)"
4,N.C. State,Texas,66.677328,"(47.2583872532297, 85.74360609003529)",79.15,"(59.58906284750917, 98.25523849430033)"
5,N.C. State,Florida State,82.207192,"(62.8661142970186, 101.53363657163501)",73.99,"(54.16249654140884, 93.23483996962798)"
